In [ ]:
%pip install -qr requirements.txt

In [23]:
from ouster import client
from ouster import pcap
from contextlib import closing
from os import listdir
from os.path import join, isdir, abspath
from pathlib import Path
import subprocess
import shutil

from tqdm import tqdm
import cv2
import numpy as np
from joblib import Parallel, delayed

In [24]:
new_folder_name = "split_files" #Must Match the name in rename_pcap notebook

In [33]:
def parallel_dir_break_up(base,break_up_fun):
    
    all_pcap_files = []
    all_meta_files = []
    all_save_paths = []

    def recurse_get_pcap_files(base,base_save_path=None):
        
        nonlocal all_pcap_files, all_meta_files, all_save_paths
        
        if base_save_path is None:
            base_save_path = join(abspath(base),"modified")

        files = listdir(base)

        directories = list(filter(lambda x: isdir(join(base,x)),files))

        pcap_files = list(map(lambda x: join(base,x),filter(lambda x: x.endswith(".pcap"),files)))

        meta_files = map(lambda x: str(Path(x).with_suffix(".json").resolve()),pcap_files)
        
        all_pcap_files.extend(pcap_files)
        all_meta_files.extend(meta_files)
        
        temp_save_paths = []
        
        for pcap in pcap_files:
            p = join(base_save_path,Path(pcap).stem,new_folder_name)
            
            Path(p).mkdir(parents=True,exist_ok=True)
            
            temp_save_paths.append(p)
        
        all_save_paths.extend(temp_save_paths)

        for directory in directories:
            recurse_get_pcap_files(join(base,directory),join(base_save_path,directory))
    
    recurse_get_pcap_files(base)

    if all_pcap_files:
        Parallel(n_jobs=4, prefer="threads")(delayed(break_up_fun)(pcap,meta,save_path)for pcap,meta,save_path in zip(all_pcap_files,all_meta_files,all_save_paths))


In [34]:
def break_up_pcap_windows(pcap,meta,save_path):
  print(f"Started {pcap}...")

  # print(join(save_path,Path(pcap).stem,"split_files"))
  
  shutil.copyfile(meta,join(save_path,"meta.json"))
    
  subprocess.run(["SplitCap.exe", "-r", pcap, "-o", save_path, "-s", "seconds","360","-b","100000"])

  #subprocess.run(["tcpdump", "-r", pcap, "-w", join(save_path,"small.pcap"), "-C", "2000"])
    
  print(f"Finished {pcap}...")
  

In [35]:
parallel_dir_break_up(join(".","data","New_Data","OS1-64_2022-11-21"),break_up_pcap_windows)

Started .\data\New_Data\OS1-64_2022-11-21\OS1-64_2022-11-21\Section1\2022-11-21-08-14-55_OS-1-64-122225003356-1024x20.pcap...
Started .\data\New_Data\OS1-64_2022-11-21\OS1-64_2022-11-21\Section2\2022-11-21-08-44-40_OS-1-64-122225003356-1024x20.pcap...
Started .\data\New_Data\OS1-64_2022-11-21\OS1-64_2022-11-21\Section3\2022-11-21-17-25-12_OS-1-64-122225003356-1024x20.pcap...
Finished .\data\New_Data\OS1-64_2022-11-21\OS1-64_2022-11-21\Section2\2022-11-21-08-44-40_OS-1-64-122225003356-1024x20.pcap...
Finished .\data\New_Data\OS1-64_2022-11-21\OS1-64_2022-11-21\Section1\2022-11-21-08-14-55_OS-1-64-122225003356-1024x20.pcap...
Finished .\data\New_Data\OS1-64_2022-11-21\OS1-64_2022-11-21\Section3\2022-11-21-17-25-12_OS-1-64-122225003356-1024x20.pcap...


In [ ]:
def recurse_break_up(base,break_up_fun,save_path=None):
    
    if save_path is None:
        save_path = join(abspath(base),"modified")
    
    files = listdir(base)
    
    directories = list(filter(lambda x: isdir(join(base,x)),files))
    
    pcap_files = list(filter(lambda x: x.endswith(".pcap"),files))
    
    meta_files = map(lambda x: str(Path(x).with_suffix(".json")),pcap_files)
    
    if pcap_files:
        print(f"Started {pcap_files}")
        Path(save_path).mkdir(parents=True,exist_ok=True)
        Parallel(n_jobs=4, prefer="threads")(delayed(break_up_fun)(join(base,pcap),join(base,meta),join(save_path,Path(pcap).stem,new_folder_name) )for pcap,meta in zip(pcap_files,meta_files))
        print(f"Finished {pcap_files}")
        
    for directory in directories:
        recurse_break_up(join(base,directory),break_up_fun,join(save_path,directory))

In [ ]:
recurse_break_up(join(".","data","New_Data","OS1-64_2022-11-21"),break_up_pcap_windows)

In [ ]:
prefix = "../../data/"
dates = {"SBIR_2022-10-08\\":(24,30),"SBIR_2022-10-07\\":(15,20),"SBIR_2022-09-24\\":(1,18),"SBIR_2022-10-06\\":(1,14),"SBIR_2022-10-21\\":(37,40),"SBIR_2022-10-22\\":(41,53),"SBIR_2022-10-20\\":(33,36)}

In [ ]:
## Split Pcap files in linux

In [ ]:
def break_up_pcap_linux(date,val):
  prefix = "../../new_data/" + date + f"{val}/"

  files = [f for f in listdir(prefix)]

  meta_path = prefix + files[0][:-5] + ".json"
  
  pcap_path = prefix + files[0][:-5] + ".pcap"

  save_path = prefix + files[0][:-5]

  print(f"Started {prefix}...")

  subprocess.run(["tcpdump", "-r", pcap_path, "-w", "small_file.pcap", "-C", "2000"])

  print(f"Finished {prefix}...")

In [ ]:
date = "SBIR_2022-09-24/"
start,end = dates[date]

In [ ]:
break_up_pcap_linux(date,1)

In [ ]:
Parallel(n_jobs=4, prefer="threads")(delayed(break_up_pcap_linux)(date,val) for val in range(start,end+1))

In [ ]:
##Split Pcap files in windows

In [ ]:
def break_up_pcap_windows(date,val):
  prefix = "D:\\SBIR\\" + date + f"{val}\\"

  files = [f for f in listdir(prefix)]
  
  pcap_path = prefix + files[0][:-5] + ".pcap"

  print(f"Started {pcap_path}...")

  subprocess.run(["SplitCap.exe", "-r", pcap_path, "-o", f"{prefix}small_files", "-s", "seconds","360","-b","100000"])
    
  print(f"Finished {prefix}...")
  

In [ ]:
date = "SBIR_2022-10-20\\"

In [ ]:
break_up_pcap(date,1)

In [ ]:
Parallel(n_jobs=4, prefer="threads")(delayed(break_up_pcap)(date,val)for val in range(dates[date][0],dates[date][1]+1))

<h1>Old Convert pcap to pcd</h1>

In [ ]:
def convert_to_pcd():
  prefix = "D:\\Data\\"

  files = [f for f in listdir(prefix)]
  files = list(filter(lambda x: x.endswith(".pcap"),files))
  files = list(filter(lambda x: not x.startswith("._"),files))

  print(f"Started...")
    
  def fun(fname):
    prefix = "D:\\Data\\"
    name = fname[:-5]
    out = name
    # if name.startswith("."):
    #     out = "small"+name[2:]
    # else:
    #     out = name
    pcap = prefix + name + ".pcap"
    meta = prefix + name + "_config.json"
    print(pcap,meta,f".\\{out}")
    subprocess.run(["python",f"{prefix}pcap.py" ,pcap ,meta,f"{prefix+out}" ,"pcap-to-pcd"])

  Parallel(n_jobs=4)(delayed(fun)(f)for f in files)

  print(f"Finished...")
  

In [ ]:
convert_to_pcd()

In [ ]:
%pwd